In [ ]:
!pip install streamlit pyngrok --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.1 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("342n8z2hMw3sXXnuwfqWS4TAWWa_76cShPrVPJKQykybTnHb9")


In [ ]:
%%writefile app.py
import streamlit as st
from copy import deepcopy

st.set_page_config(page_title="Tic-Tac-Toe (AI Lab 06)", layout="centered")

def evaluate(board):
    for r in range(3):
        if board[r][0]==board[r][1]==board[r][2] and board[r][0]!="_":
            return 10 if board[r][0]=="x" else -10
    for c in range(3):
        if board[0][c]==board[1][c]==board[2][c] and board[0][c]!="_":
            return 10 if board[0][c]=="x" else -10
    if board[0][0]==board[1][1]==board[2][2] and board[0][0]!="_":
        return 10 if board[0][0]=="x" else -10
    if board[0][2]==board[1][1]==board[2][0] and board[0][2]!="_":
        return 10 if board[0][2]=="x" else -10
    return 0

def is_moves_left(board):
    return any("_" in row for row in board)

def minimax(board, depth, is_maximizing):
    score = evaluate(board)
    if score in (10,-10): return score
    if not is_moves_left(board): return 0
    if is_maximizing:
        best=-1000
        for i in range(3):
            for j in range(3):
                if board[i][j]=="_":
                    board[i][j]="x"
                    best=max(best,minimax(board,depth+1,False))
                    board[i][j]="_"
        return best
    else:
        best=1000
        for i in range(3):
            for j in range(3):
                if board[i][j]=="_":
                    board[i][j]="o"
                    best=min(best,minimax(board,depth+1,True))
                    board[i][j]="_"
        return best

def find_best_move(board, ai_player="o"):
    best_val=1000 if ai_player=="o" else -1000
    best_move=(-1,-1)
    for i in range(3):
        for j in range(3):
            if board[i][j]=="_":
                board[i][j]=ai_player
                move_val=minimax(board,0,ai_player!="o")
                board[i][j]="_"
                if ai_player=="o" and move_val<best_val:
                    best_val=move_val; best_move=(i,j)
                if ai_player=="x" and move_val>best_val:
                    best_val=move_val; best_move=(i,j)
    return best_move

if "board" not in st.session_state:
    st.session_state.board=[["_"]*3 for _ in range(3)]
if "turn" not in st.session_state:
    st.session_state.turn="x"
if "game_over" not in st.session_state:
    st.session_state.game_over=False
if "winner" not in st.session_state:
    st.session_state.winner=None
if "mode" not in st.session_state:
    st.session_state.mode="Two Players"
if "player_symbol" not in st.session_state:
    st.session_state.player_symbol="x"

def reset_game():
    st.session_state.board=[["_"]*3 for _ in range(3)]
    st.session_state.turn="x"
    st.session_state.game_over=False
    st.session_state.winner=None

def check_and_update():
    score=evaluate(st.session_state.board)
    if score==10: st.session_state.game_over=True; st.session_state.winner="x"
    elif score==-10: st.session_state.game_over=True; st.session_state.winner="o"
    elif not is_moves_left(st.session_state.board): st.session_state.game_over=True; st.session_state.winner="draw"

st.title("🎮 Tic-Tac-Toe — AI Lab 06")
st.write("Evaluation: +10 for X win, −10 for O win, 0 otherwise.")

st.session_state.mode=st.radio("Mode",["Two Players","Play vs Computer"],
                               index=0 if st.session_state.mode=="Two Players" else 1)

if st.session_state.mode=="Play vs Computer":
    st.session_state.player_symbol=st.selectbox("You play as:",["x","o"],
        index=0 if st.session_state.player_symbol=="x" else 1)
st.button("Reset Game",on_click=reset_game)

st.write("Turn:", "" if st.session_state.game_over else st.session_state.turn.upper())

board_changed=False
for i in range(3):
    cols=st.columns(3)
    for j in range(3):
        cell=st.session_state.board[i][j]
        if cols[j].button(cell if cell!="_" else " ", key=f"{i}{j}"):
            if not st.session_state.game_over and cell=="_":
                if st.session_state.mode=="Two Players":
                    st.session_state.board[i][j]=st.session_state.turn
                    st.session_state.turn="o" if st.session_state.turn=="x" else "x"
                    board_changed=True
                else:
                    player=st.session_state.player_symbol
                    comp="o" if player=="x" else "x"
                    if st.session_state.turn==player:
                        st.session_state.board[i][j]=player
                        st.session_state.turn=comp
                        board_changed=True
    st.write("")

if board_changed: check_and_update()

if st.session_state.mode=="Play vs Computer" and not st.session_state.game_over:
    p=st.session_state.player_symbol; c="o" if p=="x" else "x"
    if st.session_state.turn==c:
        move=find_best_move(deepcopy(st.session_state.board), ai_player=c)
        if move!=(-1,-1):
            i,j=move
            st.session_state.board[i][j]=c
            st.session_state.turn=p
            check_and_update()
            st.experimental_rerun()

st.code("\n".join(["|".join(r) for r in st.session_state.board]))
if st.session_state.game_over:
    if st.session_state.winner=="draw": st.success("Draw!")
    else: st.success(f"Winner: {st.session_state.winner.upper()} "
                     f"(Eval {10 if st.session_state.winner=='x' else -10})")


Writing app.py


In [ ]:
from pyngrok import ngrok
import threading, time, os

def run_app():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run_app)
thread.start()

time.sleep(5)  # wait for server

public_url = ngrok.connect(8501)
print("👇 Click this link to open your app:")
print(public_url)


👇 Click this link to open your app:
NgrokTunnel: "https://subflexuously-uncallous-lainey.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!sed -i 's/st.experimental_rerun()/st.rerun()/' app.py
